## 4(1). Use Pycaret

### Imports

In [1]:
# for read data
import os
import pandas as pd

# load pycaret
from pycaret.classification import *  

### Read Data

In [2]:
path = (os.path.abspath("./input"))

feature_train = pd.read_csv(path +'/034.imp_LGBM_train.csv', encoding='cp949')
y_train = pd.read_csv(path +'/y_train.csv', encoding='cp949').group

### Data Preprocessing & Feature Engineering

In [3]:
del feature_train['custid']

In [4]:
setup = setup(pd.concat([feature_train, y_train], axis=1), target='group', session_id=123, use_gpu=True,
              preprocess=False)

,Description,Value
0,session_id,123
1,Target,group
2,Target Type,Multiclass
3,Label Encoded,"F20: 0, F30: 1, F40: 2, F50: 3, M20: 4, M30: 5, M40: 6, M50: 7"
4,Original Data,"(21587, 4657)"
5,Missing Values,0
6,Numeric Features,4656
7,Categorical Features,0
8,Transformed Train Set,"(15110, 4656)"
9,Transformed Test Set,"(6477, 4656)"


### Compare Model

In [5]:
from sklearn.metrics import log_loss
add_metric('logloss', 'LogLoss', log_loss, greater_is_better=False, target="pred_proba")

Name                                                           LogLoss
Display Name                                                   LogLoss
Score Function               <function log_loss at 0x000002B2AA0AD790>
Scorer               make_scorer(log_loss, greater_is_better=False,...
Target                                                      pred_proba
Args                                                                {}
Greater is Better                                                False
Multiclass                                                        True
Custom                                                            True
Name: logloss, dtype: object

In [6]:
# clf = create_model('lightgbm', fold=5)
clfs = compare_models(n_select=3, sort='logloss', include = ['lightgbm', 'xgboost', 'rf', 'lr'], fold=5) 
clfs

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.4291,0.7559,0.2497,0.3844,0.3764,0.2421,0.2518,1.6685,104.0120
xgboost,Extreme Gradient Boosting,0.4276,0.7469,0.2497,0.3829,0.3762,0.2414,0.2506,1.8254,126.9320
lr,Logistic Regression,0.3265,0.5084,0.1250,0.1066,0.1607,0.0000,0.0000,1.8583,1.9400
rf,Random Forest Classifier,0.3950,0.7045,0.1966,0.3257,0.3126,0.1706,0.1883,1.8592,8.4320


[LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                device='gpu', importance_type='split', learning_rate=0.1,
                max_depth=-1, min_child_samples=20, min_child_weight=0.001,
                min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=31,
                objective=None, random_state=123, reg_alpha=0.0, reg_lambda=0.0,
                silent='warn', subsample=1.0, subsample_for_bin=200000,
                subsample_freq=0),
 XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
               colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric=None, gamma=0, gpu_id=0, grow_policy='depthwise',
               importance_type=None, interaction_constraints='',
               learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
               max_delta_step=0, max_depth=6, max_leaves=0, min_child_weigh

In [ ]:
clf = create_model('xgboost', fold=5)

IntProgress(value=0, description='Processing: ', max=4)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,17:57:30
Status,. . . . . . . . . . . . . . . . . .,Fitting 5 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Extreme Gradient Boosting


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss


### Tune Model

In [ ]:
clf_tune = tune_model(m, optimize='LogLoss', search_library='optuna',early_stopping=True, n_iter=300, fold=5)

In [8]:
tuned_models = [tune_model(m, optimize='LogLoss', search_library='optuna',
                            early_stopping=True, n_iter=300, fold=5) for m in clfs]

IntProgress(value=0, description='Processing: ', max=7)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,17:57:30
Status,. . . . . . . . . . . . . . . . . .,Fitting 5 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Extreme Gradient Boosting


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss


[I 2022-06-15 16:51:49,459] Searching the best hyperparameters using 15110 samples...


[LightGBM] [Warning] feature_fraction is set=0.9884585190307693, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9884585190307693
[LightGBM] [Warning] bagging_fraction is set=0.810897843150918, subsample=1.0 will be ignored. Current value: bagging_fraction=0.810897843150918
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.9884585190307693, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9884585190307693
[LightGBM] [Warning] bagging_fraction is set=0.810897843150918, subsample=1.0 will be ignored. Current value: bagging_fraction=0.810897843150918
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.9884585190307693, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9884585190307693
[LightGBM] [Warning] bagging_fra

[LightGBM] [Warning] feature_fraction is set=0.6898205585576227, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6898205585576227
[LightGBM] [Warning] bagging_fraction is set=0.991335871366423, subsample=1.0 will be ignored. Current value: bagging_fraction=0.991335871366423
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.6898205585576227, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6898205585576227
[LightGBM] [Warning] bagging_fraction is set=0.991335871366423, subsample=1.0 will be ignored. Current value: bagging_fraction=0.991335871366423
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.6898205585576227, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6898205585576227
[LightGBM] [Warning] bagging_fra


KeyboardInterrupt



In [ ]:
tuned_models

### Ensemble Models

In [ ]:
# Averaging
# blender = blend_models(estimator_list=tuned_models, optimize='logloss', method='soft', fold=5, choose_better=True)

# Stacking
# stacker = stack_models(estimator_list=tuned_models, meta_model=create_model('rf'), optimize='logloss', fold=5, choose_better=True)

### Deploy Model

In [ ]:
import joblib
joblib.dump(lgbm_tune, path+'/Pycaret_lgbm.pkl')

In [9]:
# best_model = automl(optimize='logloss')

# pred = predict_model(best_model, data=feature_test) 
# pred